<a href="https://colab.research.google.com/github/IshantheG/Filled/blob/main/Algal_Blooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from random import seed
from random import randint
# use automatically configured elastic net algorithm
from numpy import arange
from pandas import read_csv
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import RepeatedKFold, GridSearchCV



Data Collection and Processing


In [ ]:
#loading the csv data to our pandas DataFrame
algae_data = pd.read_csv("/content/2021_WLE_Weekly_Datashare_CSV.csv-_1_.csv")

In [ ]:
algae_imp_data = algae_data.drop(columns = ['Date' ,'Wind_speed_knots', 'Wave_Ht_ft' , 'Station_Depth_m' , 'Sample_Depth_m', 'SpCond_uScm-1' , 'Arrival_Time' , 'Departure_Time' , 'Lat_deg' , 'Long_deg' , 'BeamAtten_m-1' ,	'Tra0miss_pct' ,	'DO_mgL-1' , 'PAR_uEcm-2s-1', 'Secchi_Depth_m', 'Site' , 'Sample_Depth_category', ], axis = 0)


In [ ]:
W = algae_imp_data['Extracted_CHLa_ugL-1'].round(decimals = 0)

In [ ]:
#print first 5 rows of the dataset
algae_imp_data.head()


,Sky,Temp_C,Turbidity_NTU,Particulate_Microcystin_ugL-1,Dissolved_Microcystin_ugL-1,Extracted_PC_ugL-1,Extracted_CHLa_ugL-1
0,2,19.22,10.30,0.0,0.00,3.056700,8.91774
1,2,16.65,1.86,0.0,0.00,0.133553,36.26974
2,2,21.30,37.10,0.0,0.14,18.254920,76.77930
3,2,19.96,5.67,0.0,0.00,0.776635,10.75724
4,2,20.55,39.40,0.0,0.00,10.247903,54.15930


In [ ]:
#print last 5 rows of the datatset
algae_imp_data.tail()

,Sky,Temp_C,Turbidity_NTU,Particulate_Microcystin_ugL-1,Dissolved_Microcystin_ugL-1,Extracted_PC_ugL-1,Extracted_CHLa_ugL-1
146,5,17.65,5.25,0.00,0.00,0.492174,3.411850
147,5,17.35,9.01,0.00,0.18,15.417222,13.226850
148,2,13.74,12.30,0.13,0.13,1.845166,9.489415
149,2,11.78,34.00,0.00,0.11,0.733771,2.245165
150,2,13.56,5.40,0.00,0.16,1.476077,7.344415


In [ ]:
#number of rows and colums in the dataset
algae_imp_data.shape

(151, 7)

In [ ]:
#getting some info about the data
algae_imp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Sky                            151 non-null    int64  
 1   Temp_C                         151 non-null    float64
 2   Turbidity_NTU                  151 non-null    float64
 3   Particulate_Microcystin_ugL-1  151 non-null    float64
 4   Dissolved_Microcystin_ugL-1    151 non-null    float64
 5   Extracted_PC_ugL-1             151 non-null    float64
 6   Extracted_CHLa_ugL-1           151 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 8.4 KB


In [ ]:

#checking for missing values
algae_imp_data.isnull().sum()


Sky                              0
Temp_C                           0
Turbidity_NTU                    0
Particulate_Microcystin_ugL-1    0
Dissolved_Microcystin_ugL-1      0
Extracted_PC_ugL-1               0
Extracted_CHLa_ugL-1             0
dtype: int64

In [ ]:
#statiscal measures about the data
algae_imp_data.describe()


,Sky,Temp_C,Turbidity_NTU,Particulate_Microcystin_ugL-1,Dissolved_Microcystin_ugL-1,Extracted_PC_ugL-1,Extracted_CHLa_ugL-1
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,3.536424,22.352848,9.906954,0.650397,0.135430,7.863269,22.313052
std,1.900087,3.533532,11.444194,1.241378,0.134768,12.478371,21.278869
min,0.000000,0.000000,1.130000,0.000000,0.000000,0.000000,1.052090
25%,2.000000,20.405000,3.875000,0.000000,0.000000,0.816407,8.186457
50%,5.000000,23.380000,6.820000,0.120000,0.140000,3.226893,14.494350
75%,5.000000,24.430000,11.050000,0.695000,0.220000,8.726029,28.042560
max,7.000000,27.670000,88.700000,7.280000,0.690000,74.906175,117.487500


In [ ]:
#checking the distribuiton of target variable
algae_imp_data['Extracted_CHLa_ugL-1'].value_counts()

8.826870     2
8.917740     1
14.299610    1
57.570825    1
71.025825    1
            ..
8.839870     1
9.028370     1
77.121525    1
23.204870    1
7.344415     1
Name: Extracted_CHLa_ugL-1, Length: 150, dtype: int64

Splitting the Features and Target


In [1]:
X = algae_imp_data.drop(columns = ['Extracted_CHLa_ugL-1',], axis = 1)
Y = algae_imp_data['Extracted_CHLa_ugL-1']

NameError: name 'algae_imp_data' is not defined

In [ ]:
print(Y)

0       8.917740
1      36.269740
2      76.779300
3      10.757240
4      54.159300
         ...    
146     3.411850
147    13.226850
148     9.489415
149     2.245165
150     7.344415
Name: Extracted_CHLa_ugL-1, Length: 151, dtype: float64


Splitting the Data into training and Testing


In [ ]:
X_train = X.sample(frac = 0.8, random_state = 2)
X_test = X.drop(X_train.index)
Y_train = Y.sample(frac = 0.8, random_state = 2)
Y_test = Y.drop(Y_train.index)
Y_train = Y_train.astype(int)



In [ ]:
print(Y_train)

85     17
111    30
3      10
128    18
6      17
       ..
80     19
66     47
139    10
83     44
143    11
Name: Extracted_CHLa_ugL-1, Length: 121, dtype: int64


In [ ]:
X_trainlist = X_train.values.tolist()
Y_trainlist = Y_train.values.tolist()
print(X_train, Y_train)

     Sky  Temp_C  Turbidity_NTU  Particulate_Microcystin_ugL-1  \
85     5   25.71           6.82                           0.74   
111    2   22.61           7.68                           0.86   
3      2   19.96           5.67                           0.00   
128    2   19.67           6.34                           0.22   
6      2   16.67           1.17                           0.00   
..   ...     ...            ...                            ...   
80     5   25.92           5.56                           2.39   
66     6   25.56           6.23                           1.53   
139    2   19.86           6.53                           0.00   
83     5   27.67          10.20                           5.86   
143    5   17.54           4.73                           0.00   

     Dissolved_Microcystin_ugL-1  Extracted_PC_ugL-1  
85                          0.16            5.275176  
111                         0.14           28.231623  
3                           0.00          

In [ ]:
print(X.shape, X_train.shape, X_test.shape)



(151, 6) (121, 6) (30, 6)


ML Model training

Logistic Regression


In [ ]:
modelEN = ElasticNet()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define model
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = arange(0, 1, 0.01)
# define search
search = GridSearchCV(modelEN, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_trainlist, Y_trainlist)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -11.849
Config: {'alpha': 0.0, 'l1_ratio': 0.0}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.771e+04, tolerance: 5.627e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  

In [ ]:
modelLoR = LogisticRegression(max_iter = 10000000, multi_class = 'multinomial', solver='lbfgs')
modelLoR.fit(X_trainlist, Y_trainlist)
LoRprediction = modelLoR.predict(X_trainlist)
print(LoRprediction)
print(Y_trainlist)

[  8  33  11   7  36   7  33  31   6  15   5   8  31  14  33  11   8  27
  20  15   7  38  11  15  11  52  12  31  76  56  15  11   2   7   7   9
  11   8  47  16  13  68 117   2  40   7  11   8  15   7  11   2  11  38
  11   8  11  16  68  47   7  11  11  54   8  11  45   2   8   5   8   7
  21  71   8  26   8  21  20  11   8   3  11  20  16   8   7   5   7  16
  40   3   2  30   7   8   2  11  16   7   5  20  11  15   7  20   9  11
  21  20  21  13   8   2  16  77  21  47   7  44  20]
[17, 30, 10, 18, 17, 15, 33, 31, 5, 7, 7, 8, 13, 14, 33, 11, 4, 23, 29, 15, 3, 38, 5, 20, 9, 52, 12, 16, 76, 56, 27, 11, 2, 19, 23, 14, 7, 8, 47, 19, 30, 68, 117, 6, 40, 7, 8, 27, 1, 8, 6, 15, 16, 57, 11, 82, 3, 16, 68, 28, 13, 5, 5, 54, 9, 9, 45, 2, 20, 10, 11, 18, 21, 71, 66, 26, 24, 17, 27, 36, 7, 11, 9, 20, 22, 19, 2, 15, 57, 7, 40, 3, 7, 13, 8, 6, 2, 6, 11, 9, 8, 14, 7, 5, 13, 5, 2, 6, 21, 17, 31, 15, 8, 99, 5, 77, 19, 47, 10, 44, 11]


In [ ]:
LoRaccuracy = math.sqrt(mean_squared_error(LoRprediction, Y_trainlist))
print(LoRaccuracy)

15.118266549809022


In [ ]:
#accuracy on training data
LoRTest = modelLoR.predict(X_test)
LoRAccuracy = mean_squared_error(LoRTest, Y_test)
a = math.sqrt(LoRAccuracy)
print(a)

15.421241005309716


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
modelLiR = LinearRegression()

In [ ]:
# training the Logistic Regression model with Training Data
modelLiR.fit(X_train, Y_train)

LinearRegression()

Model Evaluation

In [ ]:
LiRpred = modelLiR.predict(X_trainlist)
print(LiRpred, Y_trainlist)
LiRaccuracy = math.sqrt(mean_squared_error(LiRpred, Y_trainlist))
print(LiRaccuracy)

[20.77921611 34.8421843  10.96568122 20.51631613  8.00475079 14.46797252
 29.01972004 51.12983882 13.76826134 13.482467    9.98390379  9.68404254
 21.40572772 25.10334544 33.47129827 10.33301988 19.41342068 14.94648688
 23.66886945 14.09182747 15.13717583 50.74738087  8.73704199 11.64536843
 17.8417306  50.23270431  6.1497667  25.22415709 33.76137784 46.12746377
 15.16549772 21.9561003  19.31096768 19.54212844 19.43862779 23.1614251
 10.00255094 37.99980903 54.12127069 17.72733361 28.17850124 49.0374997
 39.18202307 21.37274671 35.86122709 13.75674792  8.77607677 12.88009012
 25.33972969 13.06884144 13.22085765 18.28955121 15.98352841 56.86820213
 14.18192376 16.7250516   5.75178194 20.49097584 48.958079   30.69923841
 12.70916669 10.12461238 10.74692449 55.82365603 10.50173552 21.19568333
 26.76577202 18.12201864 19.29929014  9.47997553 13.74551232 27.37245742
 24.31790585 82.58110332 17.62840562 32.39922633 13.82823301 21.4814893
 22.44245205  8.15081105  9.87811451 14.17974911 19.21

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
#accuracy on training data
LiRtest = modelLiR.predict(X_test)
LiRAccuracy = mean_squared_error(LiRtest, Y_test)
c = math.sqrt(LiRAccuracy)
print(c)

15.28025228561059


In [ ]:
RR = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = arange(0, 1, 0.1)
# define search
search = GridSearchCV(RR, grid)
# perform the search
results = search.fit(X_train, Y_train)
# summarize
print('MAE:', results.best_score_)
print('Config: %s' % results.best_params_)


MAE: 0.07531732534187566
Config: {'alpha': 0.0}


In [ ]:
RR1 = Ridge(alpha = 0.1)
RR1.fit(X_trainlist, Y_trainlist)
RRpred = RR1.predict(X_test)
print(math.sqrt(mean_squared_error(RRpred, Y_test)))

15.325655430372063


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but Ridge was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
training_data_accuracy = mean_squared_error(X_train_predictionlist, Y_trainlist)

print(math.sqrt(training_data_accuracy))

17.107222827873514


In [ ]:
#accuracy on training data

X_test_prediction = model.predict(X_test)
test_data_accuracy = mean_squared_error(X_test_prediction, Y_test)
c = math.sqrt(test_data_accuracy)
print(c)

15.421241005309716


In [ ]:
#accuracy on training data
RR.fit(X_trainlist, Y_trainlist)
RRtest = RR1.predict(X_test)
test_data_accuracy = mean_squared_error(RRtest, Y_test)
b = math.sqrt(test_data_accuracy)
print(b)

15.613458288506136


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but Ridge was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Predictive System

In [ ]:
seed1 = 1
x = randint(1,100)

In [ ]:
seed = x + 1
value = randint(0,150)
input_data = (X_train[value - 1:value])



test_value = (Y_trainlist[value - 1:value])
#change the input data to a numpy array

input_data_as_numpy_array = np.asarray(input_data)

#reshape the numpy array as we are predicting for only one
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

prediction = model.predict(input_data_reshaped)
if prediction == test_value:

  print('The prediction is correct: Prediction:' , prediction, 'Actual Value:', test_value)


else: strings = [str(integer) for integer in test_value]
a_string = "".join(strings)
a = int(a_string)

x = a - 6
y = a + 6

if prediction in range(x, y, 1):
  print('The prediction is close:', 'Prediction:', prediction, 'Actual Value:', test_value)

else:
  print('The prediction is not close: Prediction:' , prediction, 'Actual Value:', test_value)

The prediction is not close: Prediction: [7] Actual Value: [18]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
